# Experiment to observe "Intra-CCA fairness by Reno, Cubic and BBR at Edge, Core and Intermediate Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Generate flows

### Set experiment parameters

>cca, delay, test_duration, num_servers, flows, interval, omit

cca is the congestion control algorithm (bbr, reno or cubic)

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

test_duration is the time for which to send the iperf3 flows (10800 is used for these experiments).

num_servers is the number of ports to be opened on each receiver. 
CoreScale : 100,3000 and 500 ports
EdgeScale : 1,3 and 5 ports
IntermediateScale: 10, 30 and 50 ports

flows is the number of parallel flows to be send from each port. It is set to 1 for all the experiments.

interval is the periodic time interval to save the result by iperf3 (set to 1 in this experiment. Can be set to a higher value).

omit is the starting n seconds to ignore the iperf values.(set to 0 for all the experiments. Max value=300)

In [ ]:
cca="bbr"
delay=100
test_duration=10800
num_servers=1
flows=1
interval=1
omit=0

## Remove existing result files from the hosts

In [ ]:
for n in receiver_nodes:
    n.execute("rm -f 60*")

for n in sender_nodes:
    n.execute("rm -f sender*")

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [ ]:
for n in receiver_nodes:
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms limit 1000000")
    

### Start parallel servers on the receivers

Now start the `iperf3` flows:

In [ ]:
for i, n in enumerate(receiver_nodes):
    n.execute("sudo killall iperf3")
    n.execute_thread(f'chmod +x iperf-parallel-servers.sh && bash iperf-parallel-servers.sh '+str(num_servers))

In [ ]:
#check if the required number of ports are opened
for n in receiver_nodes:
    n.execute("ls -1 | wc -l")

### Start parallel clients on the senders

In this, the base_port is the starting address of port number on the receiver


In [ ]:
import time
                                 
for i, n in enumerate(sender_nodes):
    n.execute("sudo killall iperf3")
    n.execute_thread(f'chmod +x iperf-parallel-senders.sh && bash iperf-parallel-senders.sh 10.10.2.1'+str(i)+" "+str(num_servers)+" "+\
                     str(test_duration)+" "+cca+" "+str(flows)+" "+str(interval)+" "+str(omit))
    n.execute_thread(f'chmod +x cwn.sh && bash cwn.sh 10.10.2.1'+str(i))
                                       
time.sleep(test_duration+300)             

## Analyze the results

Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

In [ ]:

sum_bw = []
sum_sq_bw = []
count_flow = []

for n in sender_nodes:
    (sum_sen, serr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{sum+=$7}END {print sum}'")
    sum_bw.append(float(sum_sen.strip()))
    (sum_sq, sqerr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{sum+=$7*$7}END {print sum}'")
    sum_sq_bw.append(float(sum_sq.strip()))
    (ncount, ncerr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{count+=1}END {print count}'")
    count_flow.append(int(ncount.strip()))


total_tput=sum(sum_bw)
sum_sq_tput=sum(sum_sq_bw)
count_flow=sum(count_flow)

print("Sum of bandwidth is %f Kbits/sec " % total_tput)

print("Sum of square of bandwidth is %f" % sum_sq_tput)

print("Count of flows is %d" % count_flow)

jfi= (total_tput*total_tput)/(count_flow*sum_sq_tput)

print("JFI is %f" % jfi)

Save the output to the file.

In [ ]:
import csv
import sys
import os

jfi_filename='jfi.csv'
if not os.path.isfile(jfi_filename):
    with open(jfi_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header ='CCA', 'Duration of Expt(sec)', 'Base RTT(ms)', 'Total Bandwidth(Kbps)', 'Sum of sq of BW', 'Flow Count', 'JFI'
        writer.writerow(header)
    
with open(jfi_filename, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    columns = cca, test_duration, delay, total_tput, sum_sq_tput, count_flow, jfi
    writer.writerow(columns)
